In [1]:
#filename = 'trainingsdata/training_gesamt_extended_cutted_shuffled_cleaned.csv'
filename = 'trainingsdata/training_gesamt_extended_cutted_shuffled.csv'
punctuation = True
is_stopwords = True
is_stemming = False

In [2]:
import numpy as np
import re
from nltk.corpus import stopwords
import csv


# The `stop` is defined as earlier in this chapter
# Added it here for convenience, so that this section
# can be run as standalone without executing prior code
# in the directory
stop = stopwords.words('german')


def tokenizer(text):
    if(punctuation):
        text = re.sub('<[^>]*>', '', text)
        emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
        text = re.sub('[\W]+', ' ', text.lower()) +\
            ' '.join(emoticons).replace('-', '')
    if(is_stopwords):
        tokenized = [w for w in text.split() if w not in stop]
    else:
        tokenized = [w for w in text.split()]
    return tokenized
         

In [3]:
import numpy as np
import pandas as pd

def shuffle(path, rename):
    df = pd.read_csv(path, header=0, delimiter=";")
    frame = df.reindex(np.random.permutation(df.index))
    if rename == 1:
        name = path[:path.rindex('.')]
        extension = path[path.rindex('.'):]
        newPath = name + "_shuffled" + extension
        print(path, '=>', newPath)
        frame.to_csv(newPath, sep=';', index=False, header=False)
    else:
        frame.to_csv(path, sep=';', index=False, header=False)
        
#shuffle(path='trainingsdata/training_gesamt_extended_cutted.csv', rename=1)

In [4]:
from nltk.stem import SnowballStemmer 

def stem_sentence(sentence, algorithm):
    words = tokenizer(sentence)
    
    # print("\nSnowballStemmer (Porter vs. German):")
    stemming = SnowballStemmer(algorithm)
    
    # for w in words: 
        # print(" => ", w, ":", sPorter.stem(w), "|", sGerman.stem(w))
        
    newWords = map(lambda w: stemming.stem(w), words) 
        
    # i = 0
    # for w in words:
    #     words[i] = stemming.stem(w)
    #     i+=1
    # return ' '.join(words)
    
    return ' '.join(newWords)

In [5]:
def stream_docs(path):
    #with open(path, 'rb') as csvfile:
   #     csv_reader = csv.reader(csvfile, delimiter=';', quotechar='\'')
   #     for line in csv_reader:
    #        text, label = line[:-3], int(line[-2])
    #        yield text, label     
    
    with open(path, 'r', encoding='utf-8') as csv:
        #next(csv)  # skip header
        for line in csv:
            #print(line)
            text, label = line[:-3], int(line[-2])
            if(is_stemming):
                text = stem_sentence(text, 'german')
            yield text, label   

In [6]:
next(stream_docs(path=filename))

('ch hasse sarah Connor, aba dieses eine Lied (Music is the Key oda so) is eig. ganz gut hörbar... liegt aba sicha net an sarah, sondern an naturally 7',
 0)

In [7]:
def get_minibatch(doc_stream, size, i):
    docs, y = [], []
    try:
        for j in range(size):
            #print(i * size + j)
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [8]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [9]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

if Version(sklearn_version) < '0.18':
    clf = SGDClassifier(loss='log', random_state=5, n_iter=1)
else:
    clf = SGDClassifier(loss='log', random_state=5, max_iter=1)

doc_stream = stream_docs(path=filename)

In [10]:
#Only needed for progressbar
import pyprind
pbar = pyprind.ProgBar(35) #35

classes = np.array([0, 1])
for i in range(35):
    X_train, y_train = get_minibatch(doc_stream, size=100, i = i)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

C:\Users\Matthias\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [11]:
X_test, y_test = get_minibatch(doc_stream, size=500, i = 0) #900
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.838


In [12]:
clf = clf.partial_fit(X_test, y_test)

In [13]:
doc_stream = stream_docs(path='trainingsdata/german_validation.csv')

X_val, y_val = get_minibatch(doc_stream, size=27, i = 0)
X_val = vect.transform(X_val)
print('Accuracy: %.3f' % clf.score(X_val, y_val))

Accuracy: 0.593


In [14]:
predictions = clf.predict(X_val)
for p in predictions:
    print(p)

1
0
0
0
0
1
0
0
1
0
1
1
0
1
0
0
0
1
1
0
1
1
0
1
0
0
0
